In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

###  也可以通过cut方法
pandas.cut使用总结

用途

pandas.cut用来把一组数据分割成离散的区间。比如有一组年龄数据，可以使用pandas.cut将年龄数据分割成不同的年龄段并打上标签。

`pd.cut(
    x,
    bins,
    right: bool = True,
    labels=None,
    retbins: bool = False,
    precision: int = 3,
    include_lowest: bool = False,
    duplicates: str = 'raise',
)`

参数含义

- x：被切分的类数组（array-like）数据，必须是1维的（不能用DataFrame）；
- bins：bins是被切割后的区间（或者叫“桶”、“箱”、“面元”），有3中形式：一个int型的标量、标量序列（数组）或者pandas.IntervalIndex 。

    一个int型的标量
    当bins为一个int型的标量时，代表将x平分成bins份。x的范围在每侧扩展0.1%，以包括x的最大值和最小值。
    标量序列
    标量序列定义了被分割后每一个bin的区间边缘，此时x没有扩展。
    pandas.IntervalIndex
    定义要使用的精确区间。

- right：bool型参数，默认为True，表示是否包含区间右部。比如如果bins=[1,2,3]，right=True，则区间为(1,2]，(2,3]；right=False，则区间为(1,2),(2,3)。
- labels：给分割后的bins打标签，比如把年龄x分割成年龄段bins后，可以给年龄段打上诸如青年、中年的标签。labels的长度必须和划分后的区间长度相等，比如bins=[1,2,3]，划分后有2个区间(1,2]，(2,3]，则labels的长度必须为2。如果指定labels=False，则返回x中的数据在第几个bin中（从0开始）。
- retbins：bool型的参数，表示是否将分割后的bins返回，当bins为一个int型的标量时比较有用，这样可以得到划分后的区间，默认为False。
- precision：保留区间小数点的位数，默认为3.
- include_lowest：bool型的参数，表示区间的左边是开还是闭的，默认为false，也就是不包含区间左部（闭）。
- duplicates：是否允许重复区间。有两种选择：raise：不允许，drop：允许

返回值

- out：一个pandas.Categorical, Series或者ndarray类型的值，代表分区后x中的每个值在哪个bin（区间）中，如果指定了labels，则返回对应的label。
- bins：分隔后的区间，当指定retbins为True时返回。

In [ ]:
ages = np.array([1,5,10,40,36,12,58,62,77,89,100,18,20,25,30,32]) 
pd.cut(ages, 5)

可以看到ages被平分成5个区间，且区间两边都有扩展以包含最大值和最小值。

#### 将ages平分成5个区间并指定labels

In [ ]:
ages = np.array([1,5,10,40,36,12,58,62,77,89,100,18,20,25,30,32]) #年龄数据
pd.cut(ages, 5, labels=[u"婴儿",u"青年",u"中年",u"壮年",u"老年"])

#### 给ages指定区间进行分割

In [ ]:
ages = np.array([1,5,10,40,36,12,58,62,77,89,100,18,20,25,30,32]) #年龄数据
pd.cut(ages, [0,5,20,30,50,100])

In [ ]:
ages = np.array([1,5,10,40,36,12,58,62,77,89,100,18,20,25,30,32]) #年龄数据
pd.cut(ages, [0,5,20,30,50,100], labels=[u"婴儿",u"青年",u"中年",u"壮年",u"老年"])

这里不再平分ages，而是将ages分为了5个区间(0, 5],(5, 20],(20, 30],(30,50],(50,100].

#### 返回分割后的bins
- 令retbins=True即可

In [ ]:
ages = np.array([1,5,10,40,36,12,58,62,77,89,100,18,20,25,30,32]) #年龄数据
s = pd.cut(ages, [0,5,20,30,50,100], labels=[u"婴儿",u"青年",u"中年",u"壮年",u"老年"])
print(s)
print(type(s))

In [ ]:
type(s)

#### 只返回x中的数据在哪个bin
令labels=False即可

In [ ]:
ages = np.array([1,5,10,40,36,12,58,62,77,89,100,18,20,25,30,32]) #年龄数据
pd.cut(ages, [0,5,20,30,50,100], retbins=True)[0].value_counts()

## 表格视觉样式：Dataframe.style

### 1,表格样式创建

表格视觉样式：Dataframe.style → 返回pandas.Styler对象的属性，具有格式化和显示Dataframe的有用方法

样式创建：

    ① Styler.applymap：→ 按元素方式处理Dataframe
    ② Styler.apply：column- / row- / table-wise → 按行/列处理Dataframe

In [ ]:
df = pd.DataFrame(np.random.randn(10, 4), columns=['a','b','c','d'])
sty = df.style
print(sty, type(sty)) # 查看样式类型

#### 按元素处理样式 df.style.applymap( 函数 )

In [ ]:
def color_neg_red(val):
    if val < 0:
        color = 'red'
        size ="14px"
    else:
        color = 'blue'
        size ="10px"
    return 'color:%s;font-size:%s'% (color,size)
# style="color:red;font-size:12px;"
df.style.applymap(color_neg_red)
# 创建样式方法，使得小于0的数变成红色
# style.applymap() → 自动调用其中的函数

#### 按行/列处理样式 df.style.apply( 函数, axis=0按列, subset=['b','c']处理b、c列 )

In [ ]:
df

In [ ]:
# 按行/列处理样式：style.apply()
def highlight_max(s):
    # s.max()=这列的最大值 
    is_max = s == s.max() # 如果单元格中的数据等于这一列中最大值,返回True,否则范围false
    lst = [] # 定义列表保存样式
    # 循环series中每一个元素
    for v in is_max:
        if v:
            lst.append(' color: red; line-height: 1.5;">')
        else:
            lst.append('')
    return lst
df.style.apply(highlight_max, axis=0, subset=['b', 'c']) # axis：0为列，1为行，默认为0；  # subset：索引
# 创建样式方法，每列最大值填充黄色

### 2.表格显示控制

In [ ]:
df.head().style.format("{:.2%}")

按照百分数显示

In [ ]:
df = pd.DataFrame(np.random.randn(10,4),columns=['a','b','c','d'])
df 

In [ ]:
df.style.format("{:.2%}")

In [ ]:
'{:.2%}'.format(0.23124)

In [ ]:
'{:.2f}'.format(0.23124)

In [ ]:
df.head().style.format("{:.4f}")

In [ ]:
# 显示正负数
df.head().style.format("{:+.2f}")

In [ ]:
# 分列显示
df.head().style.format({'b':"{:.2%}", 'c':"{:+.3f}", 'd':"{:.3f}"})

### 3.表格样式调用

Styler内置样式调用

In [ ]:
# 定位空值
df = pd.DataFrame(np.random.rand(5, 4), columns=list('ABCD'))
df['A'][2] = np.nan
df

In [ ]:
# 定位空值
df.style.highlight_null(null_color='red')

#####  色彩映射
`df.style.background_gradient(cmap='Greens',axis =1,low=0,high=1)` 色彩映射

In [ ]:
df = pd.DataFrame(np.random.rand(10,4),columns = list('ABCD'))
df.style.background_gradient(axis =1) # cmap：颜色; # axis：映射参考，0为行，1以列

#### 条形图

In [ ]:
df = pd.DataFrame(np.random.rand(10,4),columns = list('ABCD'))
df

In [ ]:
df.style.bar(subset=['A', 'B'], color='#EED8AE', width=100) # width：最长长度在格子的占比

In [ ]:
df = pd.DataFrame(np.random.rand(10,4),columns = list('ABCD'))
df['A'][[3,2]] = np.nan
df

In [ ]:
# 分段式构建样式
df.style.bar(subset=['A', 'B'], color='#d65f5f', width=100).highlight_null(null_color='red')